In [1]:
import yfinance as yf
import numpy as np

data_NASDAQ = yf.download('^NDX', '2010-01-01', '2023-05-1')
data_NSE = yf.download('^NSEI', '2010-01-01', '2023-05-1')

# Align data based on common dates
common_dates = np.intersect1d(data_NASDAQ.index, data_NSE.index)
data_NASDAQ = data_NASDAQ.loc[common_dates]
data_NSE = data_NSE.loc[common_dates]

prices_NASDAQ = data_NASDAQ['Adj Close'].values
prices_NSE = data_NSE['Adj Close'].values

correlation_coefficient = np.corrcoef(prices_NASDAQ, prices_NSE)[0, 1]

print("Correlation Coefficient:", correlation_coefficient)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Correlation Coefficient: 0.9548767284940565


### In above code we have calculated the correlation cofficient from the data by yfinance

In [2]:
import pandas as pd
import ta

# Calculate Keltner Channel
keltner_channel = ta.volatility.KeltnerChannel(
    close=data_NASDAQ['Adj Close'],
    high=data_NASDAQ['High'],
    low=data_NASDAQ['Low']
)
data_NASDAQ['keltner_middle'] = keltner_channel.keltner_channel_mband()
data_NASDAQ['keltner_upper'] = keltner_channel.keltner_channel_hband()
data_NASDAQ['keltner_lower'] = keltner_channel.keltner_channel_lband()

# Calculate Bollinger Bands
bollinger_channels = ta.volatility.BollingerBands(close=data_NASDAQ['Adj Close'])
data_NASDAQ['bollinger_middle'] = bollinger_channels.bollinger_mavg()
data_NASDAQ['bollinger_upper'] = bollinger_channels.bollinger_hband()
data_NASDAQ['bollinger_lower'] = bollinger_channels.bollinger_lband()

data_NASDAQ['macd'] = ta.trend.MACD(close=data_NASDAQ['Adj Close']).macd()
data_NASDAQ['macd_signal'] = ta.trend.MACD(close=data_NASDAQ['Adj Close']).macd_signal()
data_NASDAQ['macd_histogram'] = ta.trend.MACD(close=data_NASDAQ['Adj Close']).macd_diff()

# Print the updated DataFrame
print(data_NASDAQ)



                    Open          High           Low         Close   
Date                                                                 
2010-01-04   1882.689941   1890.020020   1881.630005   1886.699951  \
2010-01-05   1887.280029   1891.329956   1877.520020   1888.430054   
2010-01-06   1887.910034   1893.209961   1873.900024   1878.420044   
2010-01-07   1878.160034   1881.329956   1867.619995   1876.719971   
2010-01-08   1870.900024   1893.250000   1868.550049   1892.589966   
...                  ...           ...           ...           ...   
2023-04-24  12981.709961  13044.219727  12883.769531  12969.759766   
2023-04-25  12905.089844  12933.280273  12724.240234  12725.110352   
2023-04-26  12866.639648  12929.620117  12783.419922  12806.480469   
2023-04-27  12963.209961  13175.620117  12938.500000  13160.030273   
2023-04-28  13139.349609  13247.389648  13096.940430  13245.990234   

               Adj Close      Volume  keltner_middle  keltner_upper   
Date              

### We have calculated the lower ,middle and upper band of the respective three indicators

In [3]:
keltner = (data_NASDAQ['keltner_middle'] - data_NASDAQ['keltner_middle'].shift(1)) / data_NASDAQ['keltner_middle']
keltner_upper = (data_NASDAQ['keltner_upper']  - data_NASDAQ['keltner_upper'] .shift(1)) / data_NASDAQ['keltner_upper'] 
keltner_lower = (data_NASDAQ['keltner_lower'] - data_NASDAQ['keltner_lower'].shift(1)) / data_NASDAQ['keltner_lower']
keltner_avg = (keltner_upper + keltner_lower+keltner) / 3
keltner_avg = keltner_avg.dropna()

returns_bollinger = (data_NASDAQ['bollinger_middle'] - data_NASDAQ['bollinger_middle'].shift(1)) / data_NASDAQ['bollinger_middle']
returns_upper = (data_NASDAQ['bollinger_upper']  - data_NASDAQ['bollinger_upper'] .shift(1)) / data_NASDAQ['bollinger_upper'] 
returns_lower = (data_NASDAQ['bollinger_lower'] - data_NASDAQ['bollinger_lower'].shift(1)) / data_NASDAQ['bollinger_lower']
returns_bollinger = (returns_upper + returns_lower+returns_bollinger) / 3
bollinger_avg = returns_bollinger.dropna()

returns_macd = (data_NASDAQ['macd'] - data_NASDAQ['macd'].shift(1)) / data_NASDAQ['macd']
returns_signal = (data_NASDAQ['macd_signal']  - data_NASDAQ['macd_signal'] .shift(1)) / data_NASDAQ['macd_signal'] 
returns_histogram = (data_NASDAQ['macd_histogram'] - data_NASDAQ['macd_histogram'].shift(1)) / data_NASDAQ['macd_histogram']
returns_macd = (returns_signal+returns_histogram) / 2
macd_avg = returns_macd.dropna() 







### From he upper , middle and lower bands we are inculcating information about the respective returns .Then ,taking average of three to find the overall return of the respective indicators.

In [9]:
import numpy as np

# Define the calculate_sharpe_ratio function
def calculate_sharpe_ratio(prices):
    daily_returns = np.diff(prices) / prices[:-1]
    average_daily_return = np.mean(daily_returns)
    standard_deviation = np.std(daily_returns)
    risk_free_rate = 0.0357  # Assuming a risk-free rate of 3.57%
    sharpe_ratio = (average_daily_return - risk_free_rate) / standard_deviation
    return sharpe_ratio

# Define the calculate_max_drawdown function
def calculate_max_drawdown(prices):
    cumulative_returns = (1 + prices).cumprod()
    previous_peaks = np.maximum.accumulate(cumulative_returns)
    drawdowns = (cumulative_returns - previous_peaks) / previous_peaks
    max_drawdown = np.min(drawdowns)
    return max_drawdown


# Calculate daily returns for each indicator
keltner_returns = np.diff(keltner_avg) / keltner_avg[:-1]
bollinger_returns = np.diff(bollinger_avg) / bollinger_avg[:-1]
macd_returns = np.diff(macd_avg) / macd_avg[:-1]

# Calculate cumulative returns for each indicator
keltner_cumulative_return = (1 + keltner_returns).cumprod() - 1
bollinger_cumulative_return = (1 + bollinger_returns).cumprod() - 1
macd_cumulative_return = (1 + macd_returns).cumprod() - 1

# Calculate Sharpe ratio for each indicator
keltner_sharpe_ratio = calculate_sharpe_ratio(keltner_returns)
bollinger_sharpe_ratio = calculate_sharpe_ratio(bollinger_returns)
macd_sharpe_ratio = calculate_sharpe_ratio(macd_returns)

# Calculate maximum drawdown for each indicator
keltner_max_drawdown = calculate_max_drawdown(keltner_cumulative_return)
bollinger_max_drawdown = calculate_max_drawdown(bollinger_cumulative_return)
macd_max_drawdown = calculate_max_drawdown(macd_cumulative_return)

print("Keltner Cumulative Return:", keltner_cumulative_return[-1])
print("Bollinger Cumulative Return:", bollinger_cumulative_return[-1])
print("MACD Cumulative Return:", macd_cumulative_return[-1])

print("Keltner Sharpe Ratio:", keltner_sharpe_ratio)
print("Bollinger Sharpe Ratio:", bollinger_sharpe_ratio)
print("MACD Sharpe Ratio:", macd_sharpe_ratio)

print("Keltner Maximum Drawdown:", keltner_max_drawdown)
print("Bollinger Maximum Drawdown:", bollinger_max_drawdown)
print("MACD Maximum Drawdown:", macd_max_drawdown)


Keltner Cumulative Return: -1.6344026447600113
Bollinger Cumulative Return: -1.7902717538404969
MACD Cumulative Return: 4.290076777962709
Keltner Sharpe Ratio: -0.025611910440380987
Bollinger Sharpe Ratio: 0.012736265575008773
MACD Sharpe Ratio: -0.02275108848617759
Keltner Maximum Drawdown: -1.0000030405862483
Bollinger Maximum Drawdown: -1.0047414360826366
MACD Maximum Drawdown: -inf


C:\Users\91992\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:57: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)


### With the returns calculated above I have calculated the required metrics like cumulative ratio , sharpe ratio,maximum drawdown


In [5]:
import pandas as pd

# Generate buy and sell signals using Bollinger Bands data
buy_signals = []
sell_signals = []
for i in range(1, len(data_NASDAQ)):

    if data_NASDAQ['Adj Close'][i] > data_NASDAQ['bollinger_upper'][i-1] and data_NASDAQ['Adj Close'][i-1] <= data_NASDAQ['bollinger_upper'][i-1]:
        buy_signals.append(data_NASDAQ.index[i])
    elif data_NASDAQ['Adj Close'][i] < data_NASDAQ['bollinger_lower'][i-1] and data_NASDAQ['Adj Close'][i-1] >= data_NASDAQ['bollinger_lower'][i-1]:
        sell_signals.append(data_NASDAQ.index[i])

# Record signals, dates, and returns
signal_records = []
returns_records = []

for buy_date in buy_signals:
    # Find corresponding sell signal date
    sell_date = next((date for date in sell_signals if date > buy_date), None)
    if sell_date:
        # Calculate returns based on entry and exit prices
        entry_price = data_NASDAQ.loc[buy_date]['Adj Close']
        exit_price = data_NASDAQ.loc[sell_date]['Adj Close']
        returns = (exit_price - entry_price) / entry_price

        # Record the signal, dates, and returns
        signal_records.append({
            'Buy Date': buy_date,
            'Sell Date': sell_date,
            'Returns': returns
        })
        returns_records.append(returns)

# Calculate metrics using the recorded signals and returns
cumulative_return = np.prod(1 +np.array( returns_records)) - 1
sharpe_ratio = calculate_sharpe_ratio(np.array(returns_records))
max_drawdown = calculate_max_drawdown(np.array(returns_records))

# Print the results
print("Signal Records:", pd.DataFrame(signal_records))
print("Cumulative Return:", cumulative_return)
print("Sharpe Ratio:", sharpe_ratio)
print("Max Drawdown:", max_drawdown)


Signal Records:       Buy Date  Sell Date   Returns
0   2010-03-02 2010-05-05  0.057827
1   2010-03-05 2010-05-05  0.036906
2   2010-03-09 2010-05-05  0.029915
3   2010-03-10 2010-05-05  0.021337
4   2010-03-11 2010-05-05  0.017907
..         ...        ...       ...
217 2022-07-28 2022-08-29 -0.018364
218 2022-07-29 2022-08-29 -0.035809
219 2022-08-03 2022-08-29 -0.058019
220 2022-10-25 2022-12-15 -0.027830
221 2022-11-11 2022-12-15 -0.039925

[222 rows x 3 columns]
Cumulative Return: 15.654544494660339
Sharpe Ratio: 0.02722061576249878
Max Drawdown: -0.6121208574770769


### In the above case we have generated the buy and sell signals for the bollinger curve that when when the prices goes over the upper band it shows buy signal and on contrary , if goes under the lower band it shows sell signals

In [6]:
# Generate buy and sell signals using keltner channels data
buy_signal = []
sell_signal = []
for i in range(1, len(data_NASDAQ)):
    if data_NASDAQ['Adj Close'][i] > data_NASDAQ['keltner_upper'][i-1] and data_NASDAQ['Adj Close'][i-1] <= data_NASDAQ['keltner_upper'][i-1]:
        buy_signal.append(data_NASDAQ.index[i])
    elif data_NASDAQ['Adj Close'][i] < data_NASDAQ['keltner_lower'][i-1] and data_NASDAQ['Adj Close'][i-1] >= data_NASDAQ['keltner_lower'][i-1]:
        sell_signal.append(data_NASDAQ.index[i])
        
# Record the signal, dates, and returns
signal_record = []
returns_record = []

for buy_date in buy_signal:
    # Find corresponding sell signal date
    sell_date = next((date for date in sell_signal if date > buy_date), None)
    if sell_date:
        # Calculate returns based on entry and exit prices
        entry_price = data_NASDAQ.loc[buy_date]['Adj Close']
        exit_price = data_NASDAQ.loc[sell_date]['Adj Close']
        returns = (exit_price - entry_price) / entry_price

        signal_record.append({
            'Buy Date': buy_date,
            'Sell Date': sell_date,
            'Returns': returns
        })
        returns_record.append(returns)

# Calculate metrics using the recorded signals and returns
cumulative_return = np.prod(1 + np.array(returns_record)) - 1
sharpe_ratio = calculate_sharpe_ratio(np.array(returns_record))
max_drawdown = calculate_max_drawdown(np.array(returns_record))

# Print the results
print("Signal Records:", pd.DataFrame(signal_record))
print("Cumulative Return:", cumulative_return)
print("Sharpe Ratio:", sharpe_ratio)
print("Max Drawdown:", max_drawdown)


Signal Records:       Buy Date  Sell Date   Returns
0   2010-02-24 2010-05-04  0.086322
1   2010-04-29 2010-05-04 -0.036184
2   2010-06-15 2010-06-29 -0.070314
3   2010-07-22 2010-08-12 -0.016601
4   2010-09-03 2010-11-16  0.119403
..         ...        ...       ...
199 2022-11-30 2022-12-15 -0.056927
200 2023-01-11 2023-02-24  0.049737
201 2023-01-20 2023-02-24  0.030176
202 2023-03-16 2023-04-25  0.011423
203 2023-04-13 2023-04-25 -0.029313

[204 rows x 3 columns]
Cumulative Return: 3.8850859752555635
Sharpe Ratio: -0.11646672202781229
Max Drawdown: -0.4157109118628939


In [7]:
# Generate buy and sell signals using keltner channels data
buy_signalss = []
sell_signalss = []
for i in range(1, len(data_NASDAQ)):
    if data_NASDAQ['macd'][i] >data_NASDAQ ['macd_signal'][i] and data_NASDAQ ['macd_histogram'][i] > 0 :
        buy_signalss.append(data_NASDAQ.index[i])
    elif data_NASDAQ['macd'][i] <data_NASDAQ ['macd_signal'][i] and data_NASDAQ ['macd_histogram'][i] < 0 :
        sell_signalss.append(data_NASDAQ.index[i])
    
# Record the signal, dates, and returns
signal_record = []
return_record = []
    
for buy_date in buy_signalss:
    # Find corresponding sell signal date
    sell_date = next((date for date in sell_signalss if date > buy_date), None)
    
    if sell_date:
    # Calculate returns based on entry and exit prices
        entry_price = data_NASDAQ.loc[buy_date]['Adj Close']
        exit_price = data_NASDAQ.loc[sell_date]['Adj Close']
        returns = (exit_price - entry_price) / entry_price
    
        signal_record.append({
            'Buy Date': buy_date,
            'Sell Date': sell_date,
            'Returns': returns
        })
        returns_record.append(returns)
        
returns_record = [r for r in returns_record if r != 0]

# Calculate metrics using the recorded signals and returns
cumulative_return = np.prod(1 +np.array( returns_record)) - 1
sharpe_ratio = calculate_sharpe_ratio(np.array(returns_record))
max_drawdown = calculate_max_drawdown(np.array(returns_record))

# Print the results
print("Signal Records:", pd.DataFrame(signal_record))
print("Cumulative Return:", cumulative_return)
print("Sharpe Ratio:", sharpe_ratio)
print("Max Drawdown:", max_drawdown)

Signal Records:        Buy Date  Sell Date   Returns
0    2010-02-24 2010-04-01  0.081131
1    2010-02-25 2010-04-01  0.080892
2    2010-02-26 2010-04-01  0.077463
3    2010-03-02 2010-04-01  0.058529
4    2010-03-03 2010-04-01  0.058324
...         ...        ...       ...
1674 2023-04-11 2023-04-19  0.009608
1675 2023-04-12 2023-04-19  0.018707
1676 2023-04-13 2023-04-19 -0.001577
1677 2023-04-17 2023-04-19  0.000076
1678 2023-04-18 2023-04-19 -0.000235

[1679 rows x 3 columns]
Cumulative Return: 107665.84045557328
Sharpe Ratio: -0.03915225749030491
Max Drawdown: -0.9268843787302202
